## Cálculo de cuántos números primos hay en un intervalo

In [4]:
def prime_count_vanilla_range(range_from: int, range_til: int):
    """ Returns the number of found prime numbers using range"""
    prime_count = 0
    range_from = range_from if range_from >= 2 else 2
    for num in range(range_from, range_til + 1):
        for divnum in range(2, num):
            if ((num % divnum) == 0):
                break
            else:
                prime_count += 1
    return prime_count

### Option 1: Calculate each number sequentially (spoiler: this is really inefficient)

In [6]:
import time

numbers = [(1, 60000)]

start_time = time.time()

#  Truqillo para pasarle varios parámetros a una función

for pair in numbers:
    prime_sum = prime_count_vanilla_range(*pair)

end_time = time.time()

print(f'Números primos encontrados en el intervalo {numbers}: {prime_sum}')
print(f'Tiempo total para encontrarlos: {end_time - start_time:2f}s')

Números primos encontrados en el intervalo [(1, 60000)]: 172199035
Tiempo total para encontrarlos: 99.293789s


In [7]:
%%time
%timeit -r2
numbers = [(1, 60000)]

#start_time = time.time()

#  Truqillo para pasarle varios parámetros a una función

for pair in numbers:
    prime_sum = prime_count_vanilla_range(*pair)

#end_time = time.time()

print(f'Números primos encontrados en el intervalo {numbers}: {prime_sum}')
#print(f'Tiempo total para encontrarlos: {end_time - start_time:2f}s')

Números primos encontrados en el intervalo [(1, 60000)]: 172199035
CPU times: user 1min 38s, sys: 17.7 ms, total: 1min 38s
Wall time: 1min 39s


Utilizando las funciones mágicas de IPython se puede observar que el tiempo que se tarda en realizar la función es ligeramente diferente. El motivo de esto se debe a que, al importar el paquete time, se nos mide el tiempo en que tarda en ejecutarse la función una sola vez. Por el contrario, las funciones mágicas utilizadas realizan la ejecución del programa un número determinado de veces y calculan el promedio de tiempo que toma dicha ejecución. En este caso, al especificar el modificador -r2, el programa se ejecuta 2 veces y se calcula el tiempo medio que ha tardado en ejecutarse.

### A Python Implementation Using Numba: The fastest

In [3]:
import time
import numba
from numba import njit

# The same implementation as prime_count_vanilla_range w/ @njit before
@njit
def prime_count_numba(range_from: int, range_til: int):
    """ Returns the number of found prime numbers using range"""
    prime_count = 0
    range_from = range_from if range_from >= 2 else 2
    for num in range(range_from, range_til + 1):
        for divnum in range(2, num):
            if ((num % divnum) == 0):
                break
            else:
                prime_count += 1
    return prime_count

numbers = [(1, 60000)]

start_time = time.time()

#  Truqillo para pasarle varios parámetros a una función
for pair in numbers:
    prime_sum = prime_count_numba(*pair)

end_time = time.time()

start_time = time.time()

#  Truqillo para pasarle varios parámetros a una función
for pair in numbers:
    prime_sum = prime_count_numba(*pair)

end_time = time.time()

print(f'Números primos encontrados en el intervalo {numbers}: {prime_sum}')
print(f'Tiempo total para encontrarlos: {end_time - start_time:2f}s')

Números primos encontrados en el intervalo [(1, 60000)]: 172199035
Tiempo total para encontrarlos: 2.778382s


Para poder ejecutar la función con el paquete numba y decorador @njit correctamente, es necesario que el código se compile previamente antes de ejecutarse. Esta compilación consiste en una traducción del código a lenguaje de máquina, que posteriormente se ejecutará de manera más óptima. Este primer paso tiene lugar con la primera llamada de la función. Finalmente, es necesario volver a llamar la función para que, una vez compilada con la llamada previa, se ejecute.

### Parallel 1: Using multiprocessing to parallelise

In [8]:
import multiprocessing
import time

numbers = [(1, 60000)]
intervals = [(1, 10000), (10001, 20000), (20001, 30000), (30001, 40000), (40001, 50000), (50001, 60000)]

start_time = time.time()

# Execute asynchronously with multi processing
# Using starmap as the function has several parameters

with multiprocessing.Pool(processes=6) as pool:
    result = pool.starmap(prime_count_vanilla_range, intervals)
                     
end_time = time.time()

print(list(result))

prime_sum = sum(result)

print(f'Números primos encontrados en el intervalo {numbers}: {prime_sum}')
print(f'Tiempo total para encontrarlos: {end_time - start_time:2f}s')

[5766453, 15483380, 24562880, 33560052, 41914240, 50912030]
Números primos encontrados en el intervalo [(1, 60000)]: 172199035
Tiempo total para encontrarlos: 29.126132s


### Parallel 2: Using concurrent.futures (map) to parallelise

In [9]:
import concurrent.futures
import time

numbers = [(1, 60000)]
intervals = [(1, 10000), (10001, 20000), (20001, 30000), (30001, 40000), (40001, 50000), (50001, 60000)]

# La función necesita varios parámetros 
# Truqillo para pasarle varios parámetros a una función haciendo otra función como intermedia

def run_heavy_function(params):
    return prime_count_vanilla_range(*params)

start_time = time.time()
# Execute asynchronously with multi processing

with concurrent.futures.ProcessPoolExecutor(max_workers=6) as executor:
    resul = executor.map(run_heavy_function, intervals)

end_time = time.time()

resultados = list(resul)

print(resultados)

prime_sum = sum(resultados)

print(f'Números primos encontrados en el intervalo {numbers}: {prime_sum}')
print(f'Tiempo total para encontrarlos: {end_time - start_time:2f}s')

[5766453, 15483380, 24562880, 33560052, 41914240, 50912030]
Números primos encontrados en el intervalo [(1, 60000)]: 172199035
Tiempo total para encontrarlos: 26.172790s


### Parallel 3: Using concurrent.futures (submit) to parallelise

In [10]:
!pip install boltons
import concurrent.futures
import time
from boltons import iterutils as ite

numbers = [(1, 60000)]
#intervals = [(1, 10000), (10001, 20000), (20001, 30000), (30001, 40000), (40001, 50000), (50001, 60000)]

start_time = time.time()
# Execute asynchronously with multi processing

with concurrent.futures.ProcessPoolExecutor(max_workers=6) as exe:
    jobs = []
    for frm, to in ite.chunk_ranges(input_size=60000, chunk_size=10000):
        job = exe.submit(prime_count_vanilla_range, frm, to)
        jobs.append(job)

end_time = time.time()

print([job.result() for job in jobs])

prime_sum = sum(job.result() for job in jobs)

print(f'Números primos encontrados en el intervalo {numbers}: {prime_sum}')
print(f'Tiempo total para encontrarlos: {end_time - start_time:2f}s')

[5766453, 15483380, 24562880, 33560052, 41914240, 50912030]
Números primos encontrados en el intervalo [(1, 60000)]: 172199035
Tiempo total para encontrarlos: 26.343096s


Como se puede observar, haciendo uso del paralelismo, la búsqueda de los números primos aumenta hasta 5 veces más. El hecho de que este incremento no haya sido de 6 indica que la división del trabajo en intervalos no ha sido llevada a cabo correctamente o simplemente la sincronización se traduzca en un pequeño retraso en la velocidad. Esto se puede deber a diferentes motivos. Uno de ellos puede ser el hecho de que existan recursos compartidos.

## Combinando procesamiento paralelo con numba

In [11]:
import multiprocessing
import time

numbers = [(1, 60000)]
intervals = [(1, 10000), (10001, 20000), (20001, 30000), (30001, 40000), (40001, 50000), (50001, 60000)]

start_time = time.time()

# Execute asynchronously with multi processing
# Using starmap as the function has several parameters

with multiprocessing.Pool(processes=6) as pool:
    result = pool.starmap(prime_count_numba, intervals)
                     
end_time = time.time()

print(list(result))

prime_sum = sum(result)

print(f'Números primos encontrados en el intervalo {numbers}: {prime_sum}')
print(f'Tiempo total para encontrarlos: {end_time - start_time:2f}s')

[5766453, 15483380, 24562880, 33560052, 41914240, 50912030]
Números primos encontrados en el intervalo [(1, 60000)]: 172199035
Tiempo total para encontrarlos: 1.997588s


In [12]:
import concurrent.futures
import time

numbers = [(1, 60000)]
intervals = [(1, 10000), (10001, 20000), (20001, 30000), (30001, 40000), (40001, 50000), (50001, 60000)]

# La función necesita varios parámetros 
# Truqillo para pasarle varios parámetros a una función haciendo otra función como intermedia

def run_heavy_function(params):
    return prime_count_numba(*params)

start_time = time.time()
# Execute asynchronously with multi processing

with concurrent.futures.ProcessPoolExecutor(max_workers=6) as executor:
    resul = executor.map(run_heavy_function, intervals)

end_time = time.time()

resultados = list(resul)

print(resultados)

prime_sum = sum(resultados)

print(f'Números primos encontrados en el intervalo {numbers}: {prime_sum}')
print(f'Tiempo total para encontrarlos: {end_time - start_time:2f}s')

[5766453, 15483380, 24562880, 33560052, 41914240, 50912030]
Números primos encontrados en el intervalo [(1, 60000)]: 172199035
Tiempo total para encontrarlos: 0.935403s


In [13]:
import concurrent.futures
import time
from boltons import iterutils as ite

numbers = [(1, 60000)]
intervals = [(1, 10000), (10001, 20000), (20001, 30000), (30001, 40000), (40001, 50000), (50001, 60000)]

start_time = time.time()
# Execute asynchronously with multi processing

with concurrent.futures.ProcessPoolExecutor(max_workers=6) as exe:
    jobs = []
    for frm, to in ite.chunk_ranges(input_size=60000, chunk_size=10000):
        job = exe.submit(prime_count_numba, frm, to)
        jobs.append(job)

end_time = time.time()

print([job.result() for job in jobs])

prime_sum = sum(job.result() for job in jobs)

print(f'Números primos encontrados en el intervalo {numbers}: {prime_sum}')
print(f'Tiempo total para encontrarlos: {end_time - start_time:2f}s')

[5766453, 15483380, 24562880, 33560052, 41914240, 50912030]
Números primos encontrados en el intervalo [(1, 60000)]: 172199035
Tiempo total para encontrarlos: 1.024462s


Para poder combinar la ejecución paralela con la llevada a cabo con el código numba, he sustituido la función a la que se llama por esta última. Como resultado, se tiene que en ambas implementaciones el tiempo que tarda en ejecutar el programa se reduce drásticamente, siendo en los dos últimos casos de apenas 1 segundo.